In [1]:
%reset -f

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext Cython

In [3]:
%reload_ext Cython

In [4]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [5]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
from data_functions import make_date_to_MYR, make_MYR_to_date
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )
import data_process
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import models, layers
import warnings
warnings.filterwarnings('ignore')
from utils import statistics_first_month, p_val, stat_matrices 
from path import col_seq_can_all, col_seq_can 
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [6]:
%config Completer.use_jedi = False

### Load pre matching data

In [7]:
pp_str = 'Pre_Matching_DATA/'
Path("./"+pp_str).mkdir(exist_ok=True)

In [8]:
treated = pd.read_pickle(pp_str+'treated2Past2.p')
controls = pd.read_pickle(pp_str+'controlsPast.p')
controls  = controls[controls['HasPast'] == 1].reset_index(drop = True)

In [9]:
random = pd.read_pickle(pp_str+'random.p')

In [10]:
random.shape, random.PERS_ID.unique().shape

In [11]:
del random

In [12]:
treated.shape, treated.MYR.min(), treated.MYR.max()

((22364, 92), 201710, 202107)

In [13]:
controls.shape, controls.MYR.min(), controls.MYR.max()

((6731029, 92), 201707, 202107)

### Random Analysis

In [14]:
def generate_hybrid_random(treated = None, 
                          controls = None, 
                          hier_cols = None,
                          hier_range = None,
                          neighbors_per_treated = 10,
                          onset = 'CC_DIABETES',
                          onset_val = 0, 
                          exact_match = True,
                          exact_match_dict = None,
                          restrict_time = True):
    
    nn = neighbors_per_treated
    tr = treated.copy()
    cr = controls.copy()
    
    #we are interested in people with 0 onset at the matching date
    if onset != "":
        cr = cr[cr[onset] == onset_val].reset_index(drop = True)
        print("Offset Filtered")
    
    print('After offset filtering control ids are:{} before they were:{}'.format(len(cr.PERS_ID.unique()), len(controls.PERS_ID.unique())))
    
    tr = tr.sort_values(by = ['PERS_ID', 'MYR']).reset_index(drop = True)
    
    #get all treated in their first date of data
    tr2 = tr.groupby('PERS_ID').apply(lambda x: pd.Series(x.iloc[0])).reset_index(drop = True)
    
    gateU = False
    
    match_count  = 0
    gg = True
    
    for i, (index, row) in tqdm(enumerate(tr2.iterrows())):
        
        nn = neighbors_per_treated
        month = row.MYR
        age = row.AGE_AT_MIDMONTH
        idi = row.PERS_ID
        gender = row.MBR_GNDR
        
        #filter the controls by month. gender and age
        if restrict_time:
            filteri = ((cr.MYR.isin([month])) & 
                   (cr.AGE_AT_MIDMONTH <= age+2) & (cr.AGE_AT_MIDMONTH >= age-2)&(cr.MBR_GNDR == gender))
        
        #exact matching filtering
        if exact_match_dict:
            for col,vals in exact_match_dict.items():
                if gg:
                    print(col, vals)
                row_val = row[col]
                if vals:
                    if row[col] in vals:
                        filteri = filteri & (cr[col] == row_val)
                else:
                    filteri = filteri & (cr[col] == row_val)
        gg = False
        
        #hierarchical matching filtering
        if not(hier_cols is None):
            for k, (col, val) in enumerate(zip(hier_cols, hier_range)):
                ref = row[col]
                filteri = filteri &( (cr[col] >= ref-val ) &(cr[col] <= ref+val))
                                    
        cr_i = cr[filteri].reset_index(drop = True)
        
        if len(cr_i) < neighbors_per_treated:
            nn = min(neighbors_per_treated, len(cr_i))
        
        N5_random_samples = cr_i.sample(n=nn)
        filteri = (controls.MYR >= month) & (controls.PERS_ID.isin(N5_random_samples.PERS_ID.unique()))
        N5_matched_controls = controls[filteri].reset_index(drop=True)
        
        #merge and concat
        if i==0:
            five_samples = N5_random_samples
            matched_controls =  N5_matched_controls
            
        else:
            five_samples = pd.concat([five_samples, N5_random_samples], axis=0)
            matched_controls = pd.concat([matched_controls, N5_matched_controls], axis=0)
        
    
    return five_samples, matched_controls

In [15]:
hier_cols = ['IP_BIN6', 'ER_BIN6']
hier_range = [1,1]
exact_match_dict = {'IP_BIN6':[0,1,2], 'ER_BIN6':[0,1,2], 'LOB':None}
random_func_args = {'hier_cols':hier_cols, 'hier_range':hier_range, 'neighbors_per_treated': 5,
                    'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': True, 'exact_match_dict': exact_match_dict}
matched0, matched_data0 = generate_hybrid_random(treated.copy(), controls.copy(), **random_func_args)

Offset Filtered
After offset filtering control ids are:269017 before they were:299160


0it [00:00, ?it/s]

IP_BIN6 [0, 1, 2]
ER_BIN6 [0, 1, 2]
LOB None


1367it [08:30,  2.68it/s]


In [16]:
matched0.shape, matched_data0.shape

### Result Base

In [17]:
data_base = "./Final/Data/Random_ipfilter_g/"
result_base = "./Final/results/Random_ipfilter_g/"
stats_base = "./Final/results/Random_ipfilter_g/stats_matching/"
Path(data_base).mkdir(exist_ok=True)
Path(result_base).mkdir(exist_ok=True)
Path(stats_base).mkdir(exist_ok=True)

In [18]:
matched0.to_pickle(data_base+"matched0.p")
matched_data0.to_pickle(data_base+"final_random_controls.p")

### Statistics

In [20]:
all_columns = list(set(col for col in controls.columns)-set(['PERS_ID', 'MYR','MEDICAID_LOB', 'HasPast']))

In [21]:
random_desc = statistics_first_month(matched_data0, all_columns)

In [23]:
random_descA = matched_data0[all_columns].describe().loc[['mean', 'std', 'count']].T
random_descA['count'] = random_descA['count'].astype(int)

In [25]:
random_desc.to_csv(stats_base+"first_allcols.csv")

In [26]:
random_descA.to_csv(stats_base+"alltime_allcols.csv")

In [28]:
!aws s3 cp ./Final/Data/Random_ipfilter_g s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/CanaryPaper/Random_ipfilter_g --recursive --acl bucket-owner-full-control

upload: Final/Data/Random_ipfilter_g/matched0.p to s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/CanaryPaper/Random_ipfilter_g/matched0.p
upload: Final/Data/Random_ipfilter_g/final_random_controls.p to s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/CanaryPaper/Random_ipfilter_g/final_random_controls.p
